In [2]:
!pip install mysql-connector-python
!pip install nbconvert

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [6]:
import pandas as pd
import mysql.connector
from datetime import datetime, timedelta

def predict_weight(item, category, df):
    item_data = df[(df['name'] == item) & (df['category'] == category)]
    
    daily_avg_weight = item_data['weight'].mean()
    
    return daily_avg_weight

db_connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="zerowaste"
)

query = "SELECT name, category, weight, created_at FROM waste WHERE created_at >= CURDATE() - INTERVAL 7 DAY"
df = pd.read_sql(query, db_connection)

db_connection.close()

df['created_at'] = pd.to_datetime(df['created_at'])

df['date'] = df['created_at'].dt.date

recent_data = df[df['date'] >= (datetime.now() - timedelta(days=7)).date()]

predictions = {}
for (item, category) in recent_data[['name', 'category']].drop_duplicates().itertuples(index=False):
    predicted_weight = predict_weight(item, category, recent_data)
    predictions[(item, category)] = predicted_weight

print("Predicted weights for the next day:")
for (item, category), weight in predictions.items():
    print(f"{category.title()} - {item}: {weight:.2f} kg")

with open('predictions.txt', 'w') as file:
    for (item, category), weight in predictions.items():
        file.write(f"{category.title()} - {item}: {weight:.2f} kg for the next day\n")


Predicted weights for the next day:
Food - Expired Canned Goods: 90.00 kg
Waste - Some Waste: 106.00 kg
Food - Organic Apple Waste: 185.00 kg


C:\Users\ASUS\AppData\Local\Temp\ipykernel_17280\3388195818.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, db_connection)


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
